In [1]:
# Python standard library
import csv
import sys

# Scientific computing
import numpy as np
import scipy
import scipy.interpolate

import matplotlib.pyplot as plt
%matplotlib inline

# Pore stats
sys.path.append('/home/prestonh/Desktop/Research/pore_stats/pore_stats/oi')
import optical_imaging as oi
import oi_file

In [2]:
date = '10-10-2017'
particle = 'HCT-116_1'
channel = '10-20-10_1'
file_index = '0'

base_path = oi_file.data_base_path

oi_vid_file_path = base_path + date + '/' + particle + '/' + channel + '/oi/bin/test_camera_' + file_index
oi_events_file_path = base_path + date + '/' + particle + '/' + channel + '/oi/events/test_camera_' + file_index + '_events.json'

In [3]:
# Load video
oi_res_x = 384
oi_res_y = 112
oi_fps = 250000.

oi_vid = oi_file.Video(oi_vid_file_path, oi_res_x, oi_res_y, oi_fps)

# Load events
oi_events = oi_file.open_event_file_json(oi_events_file_path)

/home/prestonh/Desktop/Research/pore_stats/pore_stats/oi/optical_imaging.py:75: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if pixels != None:


In [4]:
reload(oi)

template_index = 100

template_frame = oi_vid.get_frame(template_index)

stage_file_path = base_path + date + particle + channel + 'oi/stage/stage_' + file_index
cs = oi.load_stage_file(stage_file_path)

c0 = cs[0]
c1 = cs[1]
c2 = cs[2]
c3 = cs[3]

oi_stage = oi.Stage(template_frame, c0, c1, c2, c3)
oi_stage.plot_stage()

IOError: [Errno 2] No such file or directory: '/home/prestonh/Desktop/Research/cancer_cells/data/10-10-2017HCT-116_110-20-10_1oi/stage/stage_0'

In [ ]:
oi_ellipse_base_path = '../data/'
oi_ellipse_path = oi_ellipse_base_path + date + particle + channel + file_index + '/ellipses'

ellipsess = []


with open(oi_ellipse_path, 'r') as file_handle:
    file_reader = csv.reader(file_handle, delimiter = '\t')
    
    # Skip header
    header_length = 2
    for i in range(header_length):
        next(file_reader)
        
        
    for row in file_reader:
        
        if row[0] == 'event #':
            # New event
            ellipsess.append([])
            continue
            
        else:
            ellipse = []
            for ele in row:
                try:
                    ellipse.append(float(ele))
                except:
                    ellipse.append(0)
                    
            ellipsess[-1].append(ellipse)

In [ ]:
# Load keep indices
filter_file_path = '../data/' + date + particle + channel + file_index + '/filter'

with open(filter_file_path, 'r') as filter_file_handle:
    filter_file_reader = csv.reader(filter_file_handle, delimiter = '\t')
    for row in filter_file_reader:
        if row[0] == 'events':
            keep_indices = [int(ele) for ele in row[1:]]

print 'loaded keep indices (', len(keep_indices), 'events total):\n', keep_indices

In [ ]:
filtered_oi_events = [oi_events[i] for i in range(len(oi_events)) if i in keep_indices]
filtered_ellipsess = [ellipsess[i] for i in range(len(ellipsess)) if i in keep_indices]

In [ ]:
oi_events = filtered_oi_events
ellipsess = filtered_ellipsess

In [ ]:
print len(ellipsess)
print len(oi_events)

In [ ]:
calculations_file_path = '../data/' + date + particle + channel + file_index + '/calculations'



with open(calculations_file_path, 'r') as file_handle:
    file_reader = csv.reader(file_handle, delimiter = ',')
    header = file_reader.next()
    
calculations = np.genfromtxt(calculations_file_path, delimiter = ',', skip_header = 1)    

header_dict = {header[i]:i for i in range(len(header))}

In [ ]:
save = True

In [ ]:
base_path = '/home/prestonh/Desktop/Research/cancer_cells/analysis/data/'

num_files = 2
dates = ['10-10-2017']*num_files
particles = ['293-T_0']*num_files
channels = ['10-20-10_1', '10-20-10_3']
indices = ['0']*2


calculations_file_paths = [base_path + dates[i] + '/' + particles[i] + '/' + channels[i] + '/' + indices[i] + '/' + 'calculations' for i in range(num_files)]



with open(calculations_file_path, 'r') as file_handle:
    file_reader = csv.reader(file_handle, delimiter = ',')
    header = file_reader.next()
    
header_dict = {header[i]:i for i in range(len(header))}
    
    
calculationss = []
for i in range(num_files):
    calculations = np.genfromtxt(calculations_file_paths[i], delimiter = ',', skip_header = 1)    
    calculationss.append(calculations)
    
num_events = []
for i in range(num_files):
    num_events.append(int(np.max(calculationss[i][:,0])))
    


In [ ]:
avg_radius_pixss = []
avg_radius_umss = []

outside_aspectss = []
narrow_aspectss = []
cavity_aspectss = []

for i in range(num_files):
    print i
    
    avg_radius_pixs = []
    avg_radius_ums = []
    
    outside_aspects = []
    narrow_aspects = []
    cavity_aspects = []
    
    calculations = np.copy(calculationss[i])

    
    
    for j in range(num_events[i]):

        event_calculations = np.copy(calculations[calculations[:,header_dict['event_num']] == j])



        xc_ums = np.copy(event_calculations[:,header_dict['xc_um']])
        aspects = np.copy(event_calculations[:,header_dict['aspect-ratio']])
        radii_pix = np.copy(event_calculations[:,header_dict['r_pix']])
        radii_um = np.copy(event_calculations[:,header_dict['r_um']])


        avg_radius_pix = radii_pix[(xc_ums > -50) & (xc_ums < 0)][0]
        avg_radius_um = radii_um[(xc_ums > -50) & (xc_ums < 0)][0]
        outside_aspect = np.mean(aspects[(xc_ums > -50) & (xc_ums < 0)])
        narrow_aspect = np.mean(aspects[(xc_ums > 10) & (xc_ums < 40)])
        cavity_aspect = np.mean(aspects[(xc_ums > 60) & (xc_ums < 90)])


        avg_radius_pixs.append(avg_radius_pix)
        avg_radius_ums.append(avg_radius_um)
        outside_aspects.append(outside_aspect)
        narrow_aspects.append(narrow_aspect)
        cavity_aspects.append(cavity_aspect)



        
    avg_radius_pixss.append(avg_radius_pixs)
    avg_radius_umss.append(avg_radius_ums)
    
    outside_aspectss.append(outside_aspects)
    narrow_aspectss.append(narrow_aspects)
    cavity_aspectss.append(cavity_aspects)

In [ ]:
for i in range(num_files):
    avg_radius_pixss[i] = np.array(avg_radius_pixss[i])
    avg_radius_umss[i] = np.array(avg_radius_umss[i])
    
    outside_aspectss[i] = np.array(outside_aspectss[i])
    narrow_aspectss[i] = np.array(narrow_aspectss[i])
    cavity_aspectss[i] = np.array(cavity_aspectss[i])

In [ ]:
for i in range(num_files):
    if i == 0:
        color = 'blue'
        marker = 'x'
    else:
        color = 'red'
        marker = 'x'
    plt.scatter(avg_radius_pixss[i], narrow_aspectss[i], marker = marker, color = color, lw = 1.5)
    #sns.kdeplot(avg_radius_pixss[i], narrow_aspectss[i], cmap = 'Reds')
    
plt.show()

In [ ]:
#plt.scatter(avg_radius_ums, narrow_aspects, marker = 'o', color = 'blue')
#sns.kdeplot(avg_radius_ums, narrow_aspects, cmap = 'Blues')

plt.scatter(avg_radius_pixs, outside_aspects, marker = 'x', color = 'red')
sns.kdeplot(avg_radius_pixs, outside_aspects, cmap = 'Reds')

#plt.scatter(avg_radius_pixs, cavity_aspects, marker = 'x', color = 'red')
#sns.kdeplot(avg_radius_pixs, cavity_aspects, cmap = 'Reds')

plt.show()